<a href="https://colab.research.google.com/github/QaziSaim/Fine-Tune-Projects/blob/main/English_to_French_Encoder_Decoder_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [2]:
eng_texts, fra_texts = [], []

with open("/content/fra.txt", "r", encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

for line in lines:
    parts = line.split("\t")
    if len(parts) >= 2:   # ensure at least English + French
        eng = parts[0].strip()
        fra = parts[1].strip()
        eng_texts.append(eng)
        fra_texts.append(fra)


In [3]:
import re, string

def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    return text.strip()

eng_texts = [clean_text(t) for t in eng_texts]
fra_texts = ["<start> " + clean_text(t) + " <end>" for t in fra_texts]


In [4]:
eng_tokenizer  = Tokenizer()
eng_tokenizer.fit_on_texts(eng_texts)
eng_sequences = eng_tokenizer.texts_to_sequences(eng_texts)


In [5]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(fra_texts)
fra_sequences = fra_tokenizer.texts_to_sequences(fra_texts)

In [6]:
# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1


In [7]:
# Sequence lengths
max_eng_len = max(len(seq) for seq in eng_sequences)
max_fra_len = max(len(seq) for seq in fra_sequences)


In [8]:
# Pad sequences
encoder_input_data = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
decoder_input_data = pad_sequences(fra_sequences, maxlen=max_fra_len, padding='post')



In [9]:
decoder_target_data = np.zeros((len(fra_sequences), max_fra_len, fra_vocab_size), dtype="float32")

In [10]:
# for i, seq in enumerate(fra_sequences):
#     for t, word_id in enumerate(seq[1:]):  # skip <start>
#         decoder_target_data[i, t, word_id] = 1.0
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = 0  # padding

# Train-Test split
enc_train, enc_val, dec_in_train, dec_in_val, dec_tar_train, dec_tar_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.1
)


In [11]:
dec_in_val[10]

array([   1,   61,   36, 1498,    2,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [12]:
# -------------------
# 4. Encoder-Decoder Model
# -------------------
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_eng_len,))
enc_emb = Embedding(eng_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_fra_len,))
dec_emb_layer = Embedding(fra_vocab_size, latent_dim, mask_zero=True)   # <-- define layer only once
dec_emb = dec_emb_layer(decoder_inputs)                                # <-- use same layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 8, 256)    │  1,776,640 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 8)         │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 16, 256)   │  3,774,976 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0],  │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 16, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 14746) │  3,789,722 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,391,962 (39.64 MB)

 Trainable params: 10,391,962 (39.64 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
# model.fit(..., callbacks=[es])


In [14]:
# -------------------
# 5. Training
# -------------------
model.fit(
    [enc_train, dec_in_train], dec_tar_train,
    batch_size=64,
    epochs=100,
    validation_data=([enc_val, dec_in_val], dec_tar_val),
    callbacks=[es]
)


Epoch 1/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 85s 87ms/step - accuracy: 0.2691 - loss: 5.0279 - val_accuracy: 0.1956 - val_loss: 3.2795
Epoch 2/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 85s 93ms/step - accuracy: 0.2075 - loss: 2.9787 - val_accuracy: 0.2276 - val_loss: 2.5828
Epoch 3/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.2375 - loss: 2.2565 - val_accuracy: 0.2463 - val_loss: 2.2012
Epoch 4/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.2608 - loss: 1.7676 - val_accuracy: 0.2617 - val_loss: 1.9361
Epoch 5/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.2818 - loss: 1.4036 - val_accuracy: 0.2733 - val_loss: 1.7671
Epoch 6/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.2994 - loss: 1.1216 - val_accuracy: 0.2817 - val_loss: 1.6468
Epoch 7/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.3141 - loss: 0.9049 - val_accuracy: 0.2873 - val_loss: 1.5712
Epoch 8/100
911/911 ━━━━━━━━━━━━━━━━━━━━ 85s 94ms/step - accuracy: 0.3278 - loss: 0

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 8, 256)    │  1,776,640 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 8)         │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 16, 256)   │  3,774,976 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0],  │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 16, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 14746) │  3,789,722 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 31,175,888 (118.93 MB)

 Trainable params: 10,391,962 (39.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,783,926 (79.28 MB)

In [16]:
model.save('encoder_translation.keras')

In [17]:
import pickle
with open('eng_tokenizer.pkl','wb') as handler:
  pickle.dump(eng_tokenizer,handler)

with open('fra_tokenizer.pkl','wb') as handler:
  pickle.dump(fra_tokenizer,handler)


In [52]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# dec_emb2 = dec_emb(decoder_inputs)  # reuse embedding layer
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb, initial_state=decoder_states_inputs
)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [53]:
# ✅ Correct: use single-step input for inference
decoder_input_single = Input(shape=(1,))
dec_emb2 = dec_emb_layer(decoder_input_single)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs
)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_input_single] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [54]:
reverse_eng_index = {i: word for word, i in eng_tokenizer.word_index.items()}
reverse_fra_index = {i: word for word, i in fra_tokenizer.word_index.items()}
fra_word_index = fra_tokenizer.word_index


In [55]:
start_token = fra_tokenizer.word_index.get("<start>")
end_token = fra_tokenizer.word_index.get("<end>")

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token

    decoded_sentence = []
    stop_condition = False

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_fra_index.get(sampled_token_index, '')

        if sampled_token_index == end_token or len(decoded_sentence) > max_fra_len:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return " ".join(decoded_sentence)


In [63]:
def predict_translation(sentence):
    # Clean and tokenize
    seq = eng_tokenizer.texts_to_sequences([sentence.lower()])
    seq = pad_sequences(seq, maxlen=max_eng_len, padding='post')

    # Decode
    return decode_sequence(seq)

# Example test
print("English: Bored.")
print("Predicted French:", predict_translation("How are you?"))



English: Bored.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted French: comment vous end  end  end  end  end  end  end  end


In [22]:
model.save('eng_fra_seq.keras')

In [60]:
encoder_model.save('encoder_model.keras')

In [61]:
decoder_model.save('decoder_model.keras')

In [62]:
import json

config = {
    "max_eng_len": int(max_eng_len),
    "max_fra_len": int(max_fra_len),
    "latent_dim": int(latent_dim)
}
with open("config.json", "w") as f:
    json.dump(config, f)
